### Collab Notebook

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 68.8 

In [3]:
import random
import torch
import pandas as pd
import numpy as np
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from simpletransformers.model import TransformerModel
PATH = 'drive/My Drive/Experiment Design/'

In [4]:
df = pd.read_csv(PATH+'data/Task2.csv')
df = df.rename(columns={'id':'sentence_id'})
df = df.dropna(axis=0)
df = df.sample(frac=1).reset_index(drop=True)
le = LabelEncoder()
labels = le.fit_transform(df['category'].values)
df['labels'] = labels
df.rename({'sentence':'text'},axis=1,inplace=True)
df = df[['text','labels','type']]

n_tags = len(set(df['labels'].values))

In [5]:
# helper functions
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def split_data(data):
    split = 0.8
    train_df = data[0:int(split*len(data))]
    eval_df = data[int(split*len(data)):]

    return train_df, eval_df

def full_results(y_test,pred):

    report_total = classification_report(y_test, pred, output_dict = True)
    df_total = pd.DataFrame(report_total).transpose()
    display(df_total)

    inc_test = []
    exc_test = []
    inc_pred = []
    exc_pred = []
    for i,t in enumerate(typ_test):
        if t == 'I':
            inc_test.append(y_test[i])
            inc_pred.append(pred[i])
        else:
            exc_test.append(y_test[i])
            exc_pred.append(pred[i])

    print("For inclusions")
    report_inc = classification_report(inc_test, inc_pred, output_dict=True)
    df_inc = pd.DataFrame(report_inc).transpose()
    display(df_inc)

    print("For exclusions")
    report_exc = classification_report(exc_test, exc_pred, output_dict=True)
    df_exc = pd.DataFrame(report_exc).transpose()
    display(df_exc)

    return df_total, df_inc, df_exc

In [6]:
args = {
   'output_dir': 'output/',
   'cache_dir': 'cache/',
   'fp16': False,
   'fp16_opt_level': 'O1',
   'max_seq_length': 128,
   'train_batch_size': 8,
   'eval_batch_size': 8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 5,
   'weight_decay': 0,
   'learning_rate': 3e-5,
   'adam_epsilon': 1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,'overwrite_output_dir': True,
   'reprocess_input_data': False,
#     'num_labels' : 4
}

### Run bert 5 times

In [8]:
seeds_list = [123, 456, 789, 232, 323]
number_of_runs = 5

for run in range(number_of_runs):
    seed_value = seeds_list[run]
    set_seed(seed_value)
    train, test = split_data(df)
    print("Train Size: {} \nTest Size: {} ".format(len(train),len(test)))

    ## Train the model
    train_data = []

    for i,row in train.iterrows():
        train_data.append(list([row['text'],row['labels']]))

    train_df = pd.DataFrame(train_data)

    model_args = ClassificationArgs(num_train_epochs=1)

    model_args = ClassificationArgs(num_train_epochs=1)

    # Create a ClassificationModel
    model = ClassificationModel(
        'bert',
        'bert-base-cased',
        num_labels = n_tags,
        args = args
    )

    model = TransformerModel('bert','bert-base-cased',args=args,use_cuda=True,num_labels=n_tags)

    # Train the model
    model.train_model(train_df)

    model.save_model(output_dir=PATH+'task2-bert-run' + str(run + 1), model = model.model)

    le = LabelEncoder()
    labels = le.fit_transform(test['labels'].values)
    test['labels'] = labels
    model = ClassificationModel('bert', PATH +'task2-bert-run' + str(run + 1))
    test_data = test['text'].values
    test_data = [str(text) for text in test_data]
    pred = model.predict(test_data)[0]
    labels = test['labels'].values
    typ_test = test['type'].values
    print("Bert Task 2 Run " + str(run+1))
    df_total, df_inc, df_exc = full_results(labels,pred)
    df_total.to_csv(PATH + 'results_bert_task2/bert_metrics_total_run.csv' + str(run + 1))
    df_inc.to_csv(PATH + 'results_bert_task2/bert_metrics_inc_run.csv' + str(run + 1))
    df_exc.to_csv(PATH + 'results_bert_task2/bert_metrics_exc_run.csv' + str(run + 1))
    print("----------------------------------------------")

Train Size: 1842 
Test Size: 461 


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:611: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/231 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/231 [00:00<?, ?it/s]

...

For inclusions


,precision,recall,f1-score,support
0,1.000000,0.978723,0.989247,47.00000
1,0.666667,1.000000,0.800000,2.00000
2,0.916667,1.000000,0.956522,33.00000
3,0.916667,0.846154,0.880000,13.00000
4,0.976190,0.931818,0.953488,44.00000
5,0.967742,0.937500,0.952381,32.00000
6,1.000000,1.000000,1.000000,13.00000
7,0.833333,1.000000,0.909091,5.00000
8,0.928571,0.928571,0.928571,28.00000
9,0.928571,0.866667,0.896552,15.00000


For exclusions


,precision,recall,f1-score,support
0,0.888889,0.888889,0.888889,18.000000
1,1.000000,0.891892,0.942857,37.000000
2,0.333333,0.500000,0.400000,2.000000
3,1.000000,0.955556,0.977273,45.000000
4,0.937500,0.750000,0.833333,20.000000
5,0.791667,0.904762,0.844444,21.000000
6,0.857143,1.000000,0.923077,6.000000
7,1.000000,1.000000,1.000000,3.000000
8,0.904762,0.950000,0.926829,40.000000
9,0.863636,1.000000,0.926829,19.000000


----------------------------------------------


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(labels,pred)

In [ ]:
le.inverse_transform(model.predict(['travel with the person you love'])[0])